## Convolutional Neural Network (CNN) Example 2
This example is from the Udemy course titled "Deep Learning A-Z - Handson ANNs"
It is the result of the competition among students.
The article can be found here: https://www.udemy.com/course/deeplearning/learn/lecture/7023358#questions/2276518 

In [1]:
# import the required libraries
from tensorflow.contrib.keras.api.keras.layers import Dropout
from tensorflow.contrib.keras.api.keras.models import Sequential
from tensorflow.contrib.keras.api.keras.layers import Conv2D
from tensorflow.contrib.keras.api.keras.layers import MaxPooling2D
from tensorflow.contrib.keras.api.keras.layers import Flatten
from tensorflow.contrib.keras.api.keras.layers import Dense
from tensorflow.contrib.keras.api.keras.callbacks import Callback
from tensorflow.contrib.keras.api.keras.preprocessing.image import ImageDataGenerator
from tensorflow.contrib.keras import backend
import os.path

In [2]:
class LossHistory(Callback):
    def __init__(self):
        super().__init__()
        self.epoch_id = 0
        self.losses = ''
 
    def on_epoch_end(self, epoch, logs={}):
        self.losses += "Epoch {}: accuracy -> {:.4f}, val_accuracy -> {:.4f}\n"\
            .format(str(self.epoch_id), logs.get('acc'), logs.get('val_acc'))
        self.epoch_id += 1
 
    def on_train_begin(self, logs={}):
        self.losses += 'Training begins...\n'

In [3]:
# set directories
script_dir = os.getcwd()
training_set_path = script_dir + "/cnn_example1_dataset/training_set"
test_set_path = script_dir + "/cnn_example1_dataset/test_set"

In [4]:
# Initialising the CNN
classifier = Sequential()
 
# Step 1 - Convolution
input_size = (128, 128)
classifier.add(Conv2D(32, (3, 3), input_shape=(*input_size, 3), activation='relu'))
 
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size=(2, 2)))  # 2x2 is optimal
 
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
 
# Adding a third convolutional layer
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
 
# Step 3 - Flattening
classifier.add(Flatten())
 
# Step 4 - Full connection
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=1, activation='sigmoid'))

In [5]:
# Compiling the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
# Part 2 - Fitting the CNN to the images
batch_size = 32
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
 
test_datagen = ImageDataGenerator(rescale=1. / 255)
 
training_set = train_datagen.flow_from_directory(training_set_path,
                                                 target_size=input_size,
                                                 batch_size=batch_size,
                                                 class_mode='binary')
 
test_set = test_datagen.flow_from_directory(test_set_path,
                                            target_size=input_size,
                                            batch_size=batch_size,
                                            class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
# Create a loss history
history = LossHistory()
 
classifier.fit_generator(training_set,
                         steps_per_epoch=8000/batch_size,
                         epochs=9,
                         validation_data=test_set,
                         validation_steps=2000/batch_size,
                         workers=12,
#                         max_q_size=100,
                         callbacks=[history])
 

Epoch 1/9
250/250 [==============================] - 71s 284ms/step - loss: 0.6758 - acc: 0.5660 - val_loss: 0.6258 - val_acc: 0.6435
Epoch 2/9
250/250 [==============================] - 76s 303ms/step - loss: 0.6199 - acc: 0.6591 - val_loss: 0.6017 - val_acc: 0.6775
Epoch 3/9
250/250 [==============================] - 73s 291ms/step - loss: 0.5784 - acc: 0.7014 - val_loss: 0.6068 - val_acc: 0.6275
Epoch 4/9
250/250 [==============================] - 74s 296ms/step - loss: 0.5386 - acc: 0.7269 - val_loss: 0.4945 - val_acc: 0.7605
Epoch 5/9
250/250 [==============================] - 77s 308ms/step - loss: 0.4982 - acc: 0.7581 - val_loss: 0.4770 - val_acc: 0.7690
Epoch 6/9
250/250 [==============================] - 75s 301ms/step - loss: 0.4826 - acc: 0.7676 - val_loss: 0.4558 - val_acc: 0.7985
Epoch 7/9
250/250 [==============================] - 74s 297ms/step - loss: 0.4546 - acc: 0.7864 - val_loss: 0.4407 - val_acc: 0.7955
Epoch 8/9
250/250 [==============================] - 82s 329ms

In [ ]:
# Save model
# model_backup_path = os.path.join(script_dir, '/cnn_example1_dataset/cat_or_dogs_model.h5')
model_backup_path = script_dir + "/cnn_example1_dataset/cat_or_dogs_model.h5"
classifier.save(model_backup_path)
print("Model saved to", model_backup_path)

In [ ]:
# Save loss history to file
# loss_history_path = os.path.join(script_dir, '/cnn_example1_dataset/loss_history.log')
loss_history_path = script_dir + "/cnn_example1_dataset/loss_history.log"
myFile = open(loss_history_path, 'w+')
myFile.write(history.losses)
myFile.close()

In [ ]:
backend.clear_session()
print("The model class indices are:", training_set.class_indices)